# TSA Chapter 5: S&P 500 vs Bitcoin Volatility

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_sp500_garch/TSA_ch5_sp500_garch.ipynb)

Comparing GARCH(1,1) volatility between S&P 500 and Bitcoin.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
import yfinance as yf

# Download S&P 500 data
sp = yf.download('^GSPC', start='2000-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
sp500_returns = (sp['Close'].pct_change() * 100).dropna()
sp500_returns = pd.Series(sp500_returns.values, index=sp500_returns.index, name='returns')
print(f"S&P 500: {len(sp500_returns)} observations")
from arch import arch_model

# Fit GARCH models on S&P 500
m_g = arch_model(sp500_returns, vol='Garch', p=1, q=1, dist='normal')
r_g = m_g.fit(disp='off')
m_j = arch_model(sp500_returns, vol='Garch', p=1, o=1, q=1, dist='normal')
r_j = m_j.fit(disp='off')
m_e = arch_model(sp500_returns, vol='EGARCH', p=1, q=1, dist='normal')
r_e = m_e.fit(disp='off')
models = {'garch': r_g, 'gjr': r_j, 'egarch': r_e}
print("GARCH models fitted successfully")

In [ ]:
import yfinance as yf

# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', end='2025-12-31', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
btc_returns = btc['Return']
print(f"Bitcoin: {len(btc)} observations")

In [ ]:
from arch import arch_model

cond_vol_sp = models['garch'].conditional_volatility
m_btc = arch_model(btc_returns, vol='Garch', p=1, q=1, dist='normal')
r_btc = m_btc.fit(disp='off')
btc_vol = r_btc.conditional_volatility

common_start = max(sp500_returns.index[0], btc.index[0])
common_end = min(sp500_returns.index[-1], btc.index[-1])
sp_c = cond_vol_sp[(sp500_returns.index >= common_start) & (sp500_returns.index <= common_end)]
btc_c = btc_vol[(btc.index >= common_start) & (btc.index <= common_end)]

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(sp_c.index, sp_c.values, color=BLUE, lw=0.8, label='S&P 500', alpha=0.8)
ax.plot(btc_c.index, btc_c.values, color=ORANGE, lw=0.8, label='Bitcoin', alpha=0.8)
ax.set_xlabel('Date'); ax.set_ylabel('GARCH(1,1) Volatility (%)')
ax.set_title('Volatility Comparison: S&P 500 vs Bitcoin', fontweight='bold')
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.text(0.02, 0.98, f'Mean Volatility:\nS&P 500: {sp_c.mean():.2f}%\nBitcoin: {btc_c.mean():.2f}%',
        transform=ax.transAxes, fontsize=9, va='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
legend_bottom(ax, ncol=2, y=-0.15)
fig.tight_layout(); fig.subplots_adjust(bottom=0.15)
save_chart(fig, 'garch_sp500_volatility'); plt.show()